In [10]:
import sfmta
import os
import psycopg2
import datetime
import pandas as pd

os.environ['SHUTTLE_DB_USER'] = "postgres"
os.environ['SHUTTLE_DB_PASSWORD'] =''

connection = sfmta.db_connect()

Connection Created


## View all columns and first 5 rows from a specific table.

In [11]:
cursor = connection.cursor()
query = '''SELECT * FROM shuttle_locations LIMIT 5;'''

cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame.from_records(results)

,0,1,2,3,4,5
0,1,1,1,2017-09-19 11:50:45,"(-122.4753792,37.8073504)",None
1,5,3,3,2017-09-19 11:55:26,"(-122.39506,37.76855)",None
2,6,2,1,2017-09-19 11:49:52,"(-122.3844162,37.7549943)",None
3,1,1,1,2017-09-19 11:51:39,"(-122.4779136,37.8150464)",None
4,8,2,2,2017-09-19 11:50:00,"(-122.396498,37.745982)",None


## View all column NAMES from a table. Notice the values in column #3. There are 6 values in total (shuttle_id, tech_provider_id, shuttle_company_id, local_timestamp, location and cnn). This matches the 6 columns returned in the query above.

In [13]:
cursor = connection.cursor()
query = '''
    SELECT *
    FROM information_schema.columns
    WHERE table_schema = 'public'
    AND table_name   = 'shuttle_locations' '''

cursor.execute(query)
results = cursor.fetchall()
pd.DataFrame.from_records(results)

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,shuttle_database,public,shuttle_locations,shuttle_id,1,None,YES,integer,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES
1,shuttle_database,public,shuttle_locations,tech_provider_id,2,None,YES,integer,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES
2,shuttle_database,public,shuttle_locations,shuttle_company_id,3,None,YES,integer,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES
3,shuttle_database,public,shuttle_locations,local_timestamp,4,None,NO,timestamp without time zone,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES
4,shuttle_database,public,shuttle_locations,location,5,None,YES,point,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES
5,shuttle_database,public,shuttle_locations,cnn,6,None,YES,integer,None,None,...,NO,None,None,None,None,None,None,NEVER,None,YES


## We can then pass those column names into `pd.DataFrame.from_records(results)`.

In [18]:
cursor = connection.cursor()
query = '''SELECT * FROM shuttle_locations LIMIT 5;'''

cursor.execute(query)
results = cursor.fetchall()

columns=['shuttle_id', 'tech_provider_id', 'shuttle_company_id', 'local_timestamp', 'location', 'cnn']
pd.DataFrame.from_records(results, columns=columns).set_index('local_timestamp')

,shuttle_id,tech_provider_id,shuttle_company_id,location,cnn
local_timestamp,,,,,
2017-09-19 11:50:45,1,1,1,"(-122.4753792,37.8073504)",None
2017-09-19 11:55:26,5,3,3,"(-122.39506,37.76855)",None
2017-09-19 11:49:52,6,2,1,"(-122.3844162,37.7549943)",None
2017-09-19 11:51:39,1,1,1,"(-122.4779136,37.8150464)",None
2017-09-19 11:50:00,8,2,2,"(-122.396498,37.745982)",None


# TODO: find all relevant tables to use (e.g. add summary tables when available)

---
# Now we'll cover `SQL` joins. In many cases we'll need to join data from more than one table together. For example, in our query above we needed to get local_timestamp, latitute and longitude from the `shuttle_locations` table and join it with license plate from the `shuttles` table.

## Let's break down the full query

```SQL
SELECT local_timestamp, location[0], location[1] 
FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
ON l.shuttle_id=s.id WHERE s.vehicle_license_plate=%(plate)s
AND local_timestamp BETWEEN %(start)s::timestamp
 AND %(end)s::timestamp
ORDER BY local_timestamp;
```



### First, we only want the query to return three items:
1. `local_timestamp`
2. `location[0]` --> which is the first value in the tuple (-122.4779136,37.8150464)
3. `location[1]` --> which is the second value in teh tuple (-122.4779136,37.8150464)

```SQL
SELECT local_timestamp, location[0], location[1] 

```


### Next, we'll define the two tables we care about as well as the column we want to join on
Note:
1. We've defined the `shuttle_locations` table and renamed it as `l` in our query
2. We've defined the `shuttles` table and renamed it `s` in our query

```SQL
FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
ON l.shuttle_id=s.id
```

### Next, we'll had our filtering criteria. We've used special syntax to allow us to substitute values. This allows us to turn a query into a python function that can be easily reused. At the end, we say that we want the results ordered by time_stamp.
Note:
1. Special syntax `%(start)s`. This will take the start variable and replace it's contents here. 
2. We've added `::timetstamp` which tells the SQL engine how to interpret the value.
3. We've added `ORDER BY local_timestamp`
4. It's best practice to add a leading semicolon at the end of each query `;`

```SQL
WHERE s.vehicle_license_plate=%(plate)s
AND local_timestamp BETWEEN %(start)s::timestamp
 AND %(end)s::timestamp
ORDER BY local_timestamp;
```

### Full python code

```python
plate = 'CP96872'

# 2017-09-19 07:30:00
start = datetime.datetime(year=2017, month=9, day=19, hour=7, minute=30)

#2017-09-20 07:30:00
end = datetime.datetime(year=2017, month=9, day=20, hour=7, minute=30)

def get_points_for_shuttle(connection, plate, start, end):
    cursor = connection.cursor()
    query = '''SELECT local_timestamp, location[0], location[1] 
               FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
               ON l.shuttle_id=s.id WHERE s.vehicle_license_plate=%(plate)s
               AND local_timestamp BETWEEN %(start)s::timestamp
                 AND %(end)s::timestamp
               ORDER BY local_timestamp;'''

    cursor.execute(query, {'plate': plate, 'start':start, 'end':end,} )
    results = cursor.fetchall()
    return pd.DataFrame.from_records(results, columns=['time', 'lat', 'lng']).set_index('time')

dataframe = get_points_for_shuttle(connection=conn, plate=plate, start=start, end=end)

# Retrive top 5 rows for a quick view of the data
dataframe.head(n=5)
```